In [10]:
import boto3

# This looks for ddb table items that are missing a certain key, and adds a default value to those items.
# Fixes a problem we had that broke the delete projects endpoint because old projects didn't have an expected key.
def add_default_value(table_name, key_name, default_value):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(table_name)

    response = table.scan()
    items = response['Items']

    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    for item in items:
        if key_name not in item:
            item[key_name] = default_value
            table.put_item(Item=item)


# Usage
table_name = 'projects-prod'
key_name = 'scheduled_with_events'
default_value = []

#add_default_value(table_name, key_name, default_value)
